In [1]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(rc={'figure.figsize': (25, 10)}, font_scale=1.5)

df = pd.read_csv('/content/drive/MyDrive/ECH_2019.csv')



In [3]:
df.sample(5)

,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof
63794,2019034315,Mujer,56,Blanca,Jefe/a de hogar,Ocupados,12000,No,Actividades de los hogares en calidad de emple...,Sí,No asistió,"Sí, asistió",No asistió,No asistió,0,0,0,0
47868,2019025725,Hombre,43,Blanca,Jefe/a de hogar,Ocupados,0,Sí,Otros tipos de comercio al por menor no realiz...,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,0,0,0,0
93929,2019050539,Mujer,23,Blanca,Hijo/a solo del jefe/a,Ocupados,17000,Sí,"Comercio al por mayor de textiles, prendas de ...",Sí,"Sí, asistió","Sí, asistió",No asistió,"Sí, asistió",0,0,0,0
64754,2019034848,Mujer,56,Blanca,Esposo/a o compañero/a,Ocupados,0,No,Comercio al por menor en almacenes no especial...,Sí,No asistió,"Sí, asistió",0,No asistió,0,0,0,0
37602,2019020236,Hombre,43,Afro o negra,Jefe/a de hogar,Ocupados,70000,No,Actividades de clubes deportivos,Sí,No asistió,"Sí, asistió","Sí, asistió","Sí, asistió",0,0,0,0


In [4]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad', 'sueldo', 'hijos'])

df[mask] = df[mask].replace(no_data)

df.sample(5)



,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof
92656,2019049878,Mujer,82,Blanca,Jefe/a de hogar,Inactivo: pensionista,0,No hay dato,.,Sí,No asistió,"Sí, asistió",No hay dato,No asistió,No hay dato,No hay dato,No hay dato,No hay dato
47264,2019025422,Mujer,36,Blanca,Hijo/a de ambos,Ocupados,30000,No,Actividades de mantenimiento del orden público...,Sí,"Sí, asistió","Sí, asistió","Sí, asistió","Sí, asistió","Sí, asistió","Sí, asistió",No asistió,No asistió
5099,2019002700,Hombre,62,Blanca,Jefe/a de hogar,Ocupados,25000,No,Actividades de la administración pública en ge...,Sí,No asistió,"Sí, asistió",No asistió,No asistió,No hay dato,No hay dato,No hay dato,No hay dato
28394,2019015287,Mujer,55,Blanca,Esposo/a o compañero/a,Ocupados,14000,No,Actividades de los hogares en calidad de emple...,Sí,No asistió,"Sí, asistió","Sí, asistió",No asistió,No hay dato,No hay dato,No hay dato,No hay dato
100471,2019054094,Mujer,65,Indígena,Jefe/a de hogar,Ocupados,0,No,Otros tipos de enseñanza,Sí,No asistió,"Sí, asistió","Sí, asistió",No asistió,"Sí, asistió",No asistió,No hay dato,"Sí, asistió"


Seleccionamos personas con edad suficiente para trabajar

In [6]:
# df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]



Creamos una columna que cuenta la cantidad de hijos por grupo familiar

In [7]:
df['hijos'] = (
    df.groupby('id_hogar')['id_hogar'].transform('count') - 1
)



# CATEGORIZAR NUMERICOS

In [8]:
df['rango_sueldos'] = pd.cut(df['sueldo'], bins=[
    0,
    5000,
    10000,
    30000,
    50000,
    100000,
    float('Inf'),
    ]).astype('category')

df['rango_edades'] = pd.cut(df['edad'], bins=[
    0,
    10,
    15,
    20,
    30,
    40,
    50,
    60,
    70,
    float('Inf'),
    ]).astype('category')

# labels=['0-5000', 'yyy', 'zzz']



### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Primero encodificamos las variables categóricas

Eliminanos las columnas que no vamos a usar o que ya categorizamos

In [12]:
df = df[[
    'sexo',
    'ascendencia',
    'estado_laboral',
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'parentesco',
    'hijos',
    'rango_sueldos',
    'rango_edades',
    ]]



### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [13]:
# Feautures
X = df.drop('estado_laboral', axis=1).to_numpy()
y = df['estado_laboral']

print('X shape', X.shape, '\nY shape', y.shape)



X shape (107871, 17) 
Y shape (107871,)


In [14]:
np.unique(y)



array(['Desocupados buscan trabajo por primera vez',
       'Desocupados en seguro de paro', 'Desocupados propiamente dichos',
       'Inactivo: estudiante', 'Inactivo: jubilado', 'Inactivo: otro',
       'Inactivo: pensionista',
       'Inactivo: realiza los quehaceres del hogar', 'Inactivo: rentista',
       'Menores de 14 años', 'Ocupados'], dtype=object)

In [15]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

X = one_hot.fit_transform(X)



### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [17]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 



In [18]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(75509, 477) (75509,)
(32362, 477) (32362,)


# DEFINIMOS ALGORITMO DE REGRESIÓN LOGISTICA

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic_reg = LogisticRegression(max_iter=20000)

logistic_reg.fit(X_train, y_train)

predictions = logistic_reg.predict(X_test)



### VALOR DE CROSS VALIDATION PARA CADA FOLD

In [20]:
scores = cross_val_score(logistic_reg, X, y, cv=10)

scores



array([0.88060808, 0.88068972, 0.88152406, 0.878094  , 0.88087513,
       0.88393437, 0.88189487, 0.88004079, 0.88050431, 0.88133865])

In [21]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



Promedio de cross_validation  0.88
Score en test  0.88
Score en train  0.88


In [22]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predictions))



[[    7     0    33   109     0    47     1    32     0     0     0]
 [    1     0    29     7     8     1     0    23     0     0     0]
 [    3     0   342   159    91    53     9   334     1     0     0]
 [    0     0    27  2073     3     0     2     0     0   425     0]
 [    0     0    19     1  4494     2    95   127     0     0     0]
 [   19     0   110     0    64   112    18    77     0     4     0]
 [    0     0    41    21   854    18   229   127     0    13     0]
 [    5     0   111     0   426    32    41  1157     0     1     0]
 [    0     0    10     2    82     0     3    40     0     0     0]
 [    0     0     0    31     0     1     0     0     0  5383     0]
 [    0     0     0     0     0     0     0     0     0     0 14772]]


### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [23]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = logistic_reg.predict(X_train)

#Prediccion en Test
y_test_pred = logistic_reg.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)



% de aciertos sobre el set de entrenamiento: 0.8818286561866797
% de aciertos sobre el set de evaluación: 0.8827946356838267


### DEFINIMOS UNA PREDICCIÓN Y LA APLICAMOS SOBRE EL MODELO

In [24]:
prediction = X.toarray()

prediction = prediction[1000].reshape(1, -1)
print(logistic_reg.predict(prediction))
print(y_train[1000])



['Inactivo: estudiante']
Inactivo: estudiante


Determinamos la probabilidad de la predicción

In [25]:
probabilidades_prediccion = logistic_reg.predict_proba(prediction)
probabilidades_prediccion[:,1]



array([0.02261323])

# REPETIMOS EL EJERCICIO CON EL MODELO DE DECISSION TREE

In [26]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(100)

random_forest.fit(X_train, y_train)

forest_prediction = random_forest.predict(X_test)



In [27]:
scores = cross_val_score(random_forest, X, y, cv=10)

scores



array([0.87458287, 0.87522017, 0.87318068, 0.87410772, 0.87475665,
       0.87522017, 0.87420043, 0.87234634, 0.87568369, 0.87373691])

In [28]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



Promedio de cross_validation  0.87
Score en test  0.88
Score en train  0.88


In [29]:
#Prediccion en Train
y_train_pred = random_forest.predict(X_train)

#Prediccion en Test
y_test_pred = random_forest.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)



% de aciertos sobre el set de entrenamiento: 0.9094412586579084
% de aciertos sobre el set de evaluación: 0.8751622273036277


In [30]:
print(random_forest.predict(prediction))
print(y_train[1000])



['Inactivo: estudiante']
Inactivo: estudiante


# REPETIMOS EL EJERCICIO CON EL MODELO DE SUPPORT VECTOR MACHINE

In [31]:
from sklearn import svm

svm_model = svm.SVC(kernel='linear')

svm_model.fit(X_train, y_train)

svm_model.predict(X_test)



array(['Menores de 14 años', 'Ocupados', 'Inactivo: jubilado', ...,
       'Inactivo: jubilado', 'Ocupados',
       'Inactivo: realiza los quehaceres del hogar'], dtype=object)

In [32]:
scores = cross_val_score(svm_model, X, y, cv=10)

scores



array([0.87467557, 0.87512747, 0.87216093, 0.87438583, 0.87447854,
       0.87661074, 0.87308798, 0.87262446, 0.87522017, 0.87382961])

In [33]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(svm_model.score(X_test, y_test),2))
print('Score en train ', round(svm_model.score(X_train, y_train),2))



Promedio de cross_validation  0.87
Score en test  0.88
Score en train  0.88


In [34]:
#Prediccion en Train
y_train_pred = svm_model.predict(X_train)

#Prediccion en Test
y_test_pred = svm_model.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)



% de aciertos sobre el set de entrenamiento: 0.8804778238355693
% de aciertos sobre el set de evaluación: 0.8821457264693159


In [35]:
print(svm_model.predict(prediction))
print(y_train[1000])



['Inactivo: estudiante']
Inactivo: estudiante
